In [ ]:
import os
import csv

from BBC_scrapper_amharic import BBCScraper

In [ ]:
pages = 1
csv_file_path = "data/bbc_amharic.csv"

# Replace with the base URL of the page you want to scrape
base_url = "https://www.bbc.com/amharic/topics/c7zp57r92v5t"
prefix = "https://www.bbc.com/amharic/articles"

In [ ]:
# save the data to a csv file


def save_to_csv(news_articles, csv_file_path):
    # Save the articles to a csv file
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=news_articles[0].keys())
        writer.writeheader()
        writer.writerows(news_articles)

In [37]:
total_articles = []

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

for page_number in range(1, pages + 1):
    page_url = f"https://www.bbc.com/amharic/topics/c7zp57r92v5t?page={page_number}"
    print(page_url)

    scrapper = BBCScraper(page_url, prefix)
    news_articles = scrapper.get_articles()
    total_articles.extend(news_articles)
    print(news_articles)

    # Save the articles to a csv file
    save_to_csv(news_articles, csv_file_path)

https://www.bbc.com/amharic/topics/c7zp57r92v5t?page=1


KeyboardInterrupt: 

In [ ]:
total_articles[0]

{'url': 'https://www.bbc.com/amharic/articles/crgywn202l0o',
 'title': 'ፕሬዝዳንት ኢሳያስ የጠቀሱት “ሌላ የጦርነት አዙሪት” ማንን ያሰጋል? - BBC News አማርኛ',
 'date': None,
 'image_url': 'https://a1.api.bbc.co.uk/hit.xiti?s=598342&s2=4&p=amharic.articles.crgywn202l0o.page&x1=[urn%3Abbc%3Aoptimo%3Aasset%3Acrgywn202l0o]&x2=[responsive]&x3=[news-amharic]&x4=[am]&x7=[article]&x8=[simorgh-nojs]&x9=[%E1%8D%95%E1%88%AC%E1%8B%9D%E1%8B%B3%E1%8A%95%E1%89%B5%2520%E1%8A%A2%E1%88%B3%E1%8B%AB%E1%88%B5%2520%E1%8B%A8%E1%8C%A0%E1%89%80%E1%88%B1%E1%89%B5%2520%E2%80%9C%E1%88%8C%E1%88%8B%2520%E1%8B%A8%E1%8C%A6%E1%88%AD%E1%8A%90%E1%89%B5%2520%E1%8A%A0%E1%8B%99%E1%88%AA%E1%89%B5%E2%80%9D%2520%E1%88%9B%E1%8A%95%E1%8A%95%2520%E1%8B%AB%E1%88%B0%E1%8C%8B%E1%88%8D%3F]&x11=[2024-05-27T04%3A06%3A06.232Z]&x12=[2024-05-27T04%3A06%3A06.232Z]&x13=[Ethiopia%2Band%2BEritrea%2Bpeace%2Bagreement~Sudan~Politics~War~Eritrea~Ethiopia]&x14=[5a491dfd-e977-43df-a5c0-39a35501688f~620270e1-cd82-443d-a7e4-98d177627606~75612fa6-147c-4a43-97fa-fcf70d9cced3

### Use kafka to store the data and use faust to process the data

In [ ]:
from kafka import KafkaProducer
import json

In [ ]:
# Create a Kafka producer
# localhost:9093 is the address of the Kafka broker for faust in docker to work
KAFKA_BROKER_URL = os.environ.get('KAFKA_BROKER_URL', 'localhost:9093')
producer = KafkaProducer(bootstrap_servers=KAFKA_BROKER_URL,
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [ ]:
# models.py
import faust

'''
- This is a faust record that will be used to store the scraped data
- Faust record, which is a type of data model that Faust can use to understand the structure of your messages.
'''
class ScrapedData(faust.Record, serializer='json'):
    url: str
    title: str
    source: str

    def to_dict(self):
        return {
            'url': self.url,
            'title': self.title,
            'source': self.source,
        }

In [ ]:
# create a scrapedData object
scraped_data = ScrapedData(url=total_articles[0]['url'], title=total_articles[0]['title'], source=total_articles[0]['source'])

In [ ]:
# send the scraped data to the 'scraped_data' Kafka topic
producer.send('scraped_data', value=scraped_data.to_dict())